# Preliminary operations

In [31]:
import pandas as pd
import numpy as np
import copy
import pickle
import os
import itertools
import glob
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from math import sqrt
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.ndimage import gaussian_filter1d

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.layers import Dense, Flatten, Reshape, Dropout
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

seed = 123
tf.keras.utils.set_random_seed(seed)
df_path_dense = []
df_path_lstm = []

In [32]:
gpu = len(tf.config.list_physical_devices('GPU')) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is NOT AVAILABLE


In [33]:
%run utils.ipynb

In [34]:
%run preprocessing.ipynb

In [35]:
%run preparation.ipynb

In [36]:
%run training_dense.ipynb

In [37]:
%run training_lstm.ipynb

In [38]:
%run predict.ipynb

In [39]:
data_path = 'datasets/raw/sf_normal_final_indicators_73000.csv'
df = pd.read_csv(data_path)
df = df.drop('timestamp', axis = 1)
num_features = len(df.columns)

In [40]:
df.columns

Index(['rides_requested', 'rides_canceled', 'rides_not_served',
       'rides_accepted', 'rides_rejections', 'idle_drivers',
       'responding_drivers', 'pickup_drivers', 'on_road_drivers',
       'moving_drivers', 'pending_customers', 'avg_rejections_before_accepted',
       'avg_surge_multiplier', 'avg_expected_price_per_km',
       'avg_real_price_per_km', 'avg_diff_price_per_km',
       'avg_current_error_ride_distance', 'avg_speed_max_speed',
       'avg_speed_kmh', 'avg_diff_duration_min', 'avg_expected_ride_time_min',
       'avg_expected_meeting_time_min', 'avg_expected_total_time_min',
       'avg_ride_time_min', 'avg_meeting_time_min', 'avg_total_time_min',
       'avg_ride_length_km', 'avg_meeting_length_km', 'avg_total_length_km',
       'avg_distance_per_timestamp_km', 'avg_remaining_distance_covered'],
      dtype='object')

In [41]:
df.head(10)

,rides_requested,rides_canceled,rides_not_served,rides_accepted,rides_rejections,idle_drivers,responding_drivers,pickup_drivers,on_road_drivers,moving_drivers,...,avg_expected_meeting_time_min,avg_expected_total_time_min,avg_ride_time_min,avg_meeting_time_min,avg_total_time_min,avg_ride_length_km,avg_meeting_length_km,avg_total_length_km,avg_distance_per_timestamp_km,avg_remaining_distance_covered
0,2,0,0,0,0,85,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,0,0,84,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1,0,0,85,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,85,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,0,0,0,84,4,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2,2,2,0,0,82,6,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2,0,0,0,0,80,8,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,0,0,0,1,82,8,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3,0,0,0,0,80,11,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,0,0,0,0,80,11,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df.tail(10)

,rides_requested,rides_canceled,rides_not_served,rides_accepted,rides_rejections,idle_drivers,responding_drivers,pickup_drivers,on_road_drivers,moving_drivers,...,avg_expected_meeting_time_min,avg_expected_total_time_min,avg_ride_time_min,avg_meeting_time_min,avg_total_time_min,avg_ride_length_km,avg_meeting_length_km,avg_total_length_km,avg_distance_per_timestamp_km,avg_remaining_distance_covered
72990,1,2,0,2,0,135,7,123,475,101,...,1.870556,13.969033,10.286667,3.753333,14.040000,5.387766,1.671982,7.059748,0.008378,0.000904
72991,0,2,0,0,0,134,7,123,475,101,...,1.870556,13.969033,10.286667,3.753333,14.040000,5.387766,1.671982,7.059748,0.008383,0.000628
72992,0,6,0,0,0,134,7,123,473,101,...,2.291196,18.283125,11.994444,3.488889,15.483333,6.225053,1.467277,7.692330,0.008401,0.000836
72993,0,3,2,0,2,136,5,123,472,101,...,2.542317,23.092224,13.141667,4.516667,17.658333,6.688435,1.662085,8.350520,0.007839,0.001374
72994,2,0,0,0,0,136,7,123,471,101,...,1.935961,16.712663,11.583333,2.383333,13.966667,5.796890,0.680040,6.476930,0.007719,0.002725
72995,1,7,1,0,0,135,8,123,471,101,...,1.935961,16.712663,11.583333,2.383333,13.966667,5.796890,0.680040,6.476930,0.007728,0.001243
72996,0,0,0,1,0,135,7,124,471,101,...,1.935961,16.712663,11.583333,2.383333,13.966667,5.796890,0.680040,6.476930,0.007729,0.000055
72997,2,5,0,0,0,132,9,123,472,101,...,1.935961,16.712663,11.583333,2.383333,13.966667,5.796890,0.680040,6.476930,0.007729,0.000047
72998,0,0,0,1,0,134,8,124,471,101,...,1.935961,16.712663,11.583333,2.383333,13.966667,5.796890,0.680040,6.476930,0.007729,0.000047
72999,0,0,0,0,0,134,8,123,472,101,...,1.935961,16.712663,11.583333,2.383333,13.966667,5.796890,0.680040,6.476930,0.007729,0.000047


In [43]:
len(df)

73000

In [44]:
df.isnull().values.any()

False

# Choosing preprocessing strategy - Dense

## Preprocessing

In [16]:
ratio_values = [True, False]
smoothing_values = [True, False]
collinearity_values = [False]
log_values = [False]
aggregation_values = [100, 200, 300]  
statistics_values = [True, False]
differencing_values = [0]
seasonality_values = [True, False]
normalize_values = [True]

# Generate all combinations of parameter values
experiments = itertools.product(
    ratio_values, smoothing_values, collinearity_values, 
    log_values, aggregation_values, statistics_values, 
    differencing_values, seasonality_values, normalize_values
)

count = 1
tot_experiments = len(ratio_values) * len(collinearity_values) * len(log_values) * len(aggregation_values) * len(statistics_values) * len(differencing_values) * len(seasonality_values) * len(normalize_values)
warnings.filterwarnings("ignore", category = RuntimeWarning)
for combination in experiments:
    print(f"Experiment {count}/{tot_experiments}")
    print(f"ratio: {combination[0]}, smoothing: {combination[1]}, collinearity: {combination[2]}, log: {combination[3]}, aggregation: {combination[4]}, statistics: {combination[5]}, differencing: {combination[6]}, seasonality: {combination[7]}, normalize: {combination[8]}")
    df_path = ('df_' + str(combination[0]) + '_' + str(combination[1]) + '_' + str(combination[2]) + '_' + str(combination[3]) + '_' + str(combination[4]) + '_' + str(combination[5]) + '_' + str(combination[6]) + '_' + str(combination[7]), + '_' + str(combination[8]))
    if df_path not in df_path_dense:
        df_path_dense.append('datasets/proc/' + df_path + '.csv')
    df_ = copy.deepcopy(df)
    if df_path not in df_path_lstm and not os.path.exists('datasets/proc/' + df_path + '.csv'):
        df_proc = preprocessing(df_,
                                ratio = combination[0],
                                smoothing = combination[1],
                                collinearity = combination[2],
                                log = combination[3],
                                aggregation = combination[4],
                                statistics = combination[5],
                                differencing = combination[6],
                                seasonality = combination[7],
                                normalize = combination[8],
                                load_scaler = False,
                                save = True)
    count += 1

Experiment 1/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: True, differencing: 0, seasonality: True, normalize: True
(67600, 248)
Experiment 2/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: True, differencing: 0, seasonality: False, normalize: True
(67600, 248)
Experiment 3/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: False, differencing: 0, seasonality: True, normalize: True
(67600, 31)
Experiment 4/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: False, differencing: 0, seasonality: False, normalize: True
(67600, 31)
Experiment 5/24
ratio: True, collinearity: False, log: False, aggregation: 200, statistics: True, differencing: 0, seasonality: True, normalize: True
(67600, 248)
Experiment 6/24
ratio: True, collinearity: False, log: False, aggregation: 200, statistics: True, differencing: 0, seasonality: False, normalize: True
(67600, 248)
Experiment 7/24
rat

## Preparation

In [17]:
pattern = os.path.join('datasets/proc', 'df*.csv')
csv_files = glob.glob(pattern)
data_prep_dict = {}

count = 1
for file_path in csv_files:
    if file_path in df_path_dense:
        print(str(count) + ': ' + file_path)
        filename = os.path.basename(file_path).replace('.csv', '')
        data_prep =  pd.read_csv(file_path)
        data_prep_dict[filename] = preparation(data_prep,
                                               window_size = 0,
                                               overlap = 0,
                                               val_hours = 1.5)
        count+=1

1: datasets/df_True_False_False_200_False_0_False_True.csv
(62200, 31)
(5400, 31)
2: datasets/df_False_False_False_100_False_0_True_True.csv
(62200, 31)
(5400, 31)
3: datasets/df_True_False_False_300_False_0_True_True.csv
(62200, 31)
(5400, 31)
4: datasets/df_True_False_False_200_True_0_False_True.csv
(62200, 248)
(5400, 248)
5: datasets/df_True_False_False_100_True_0_False_True.csv
(62200, 248)
(5400, 248)
6: datasets/df_False_False_False_300_True_0_False_True.csv
(62200, 248)
(5400, 248)
7: datasets/df_False_False_False_200_False_0_True_True.csv
(62200, 31)
(5400, 31)
8: datasets/df_False_False_False_100_False_0_False_True.csv
(62200, 31)
(5400, 31)
9: datasets/df_True_False_False_300_False_0_False_True.csv
(62200, 31)
(5400, 31)
10: datasets/df_False_False_False_100_True_0_True_True.csv
(62200, 248)
(5400, 248)
11: datasets/df_False_False_False_100_True_0_False_True.csv
(62200, 248)
(5400, 248)
12: datasets/df_True_False_False_300_True_0_False_True.csv
(62200, 248)
(5400, 248)
13: d

## Training

Here we use the standard Dense model to choose the preprocessing strategy. We may do the same for the LSTM.

In [19]:
mae_list = []
rmse_list = []
mape_list = []
name_list = []

csv_file_path = 'processing_metrics_dense.csv'

for name, data_prep in data_prep_dict.items():
    # Training
    history, model = training_dense(data_prep[0],
                                    data_prep[1],
                                    n_layers_encoder = 1,
                                    n_layers_decoder = 1,
                                    hidden_units = [64, 32, 32, 64],
                                    activation_hidden = 'relu',
                                    activation_dense = 'sigmoid',
                                    dropout_rate = 0.2,
                                    learning_rate = 0.001,
                                    n_epochs = 500,
                                    metric = 'mse',
                                    batch_size = 32,
                                    plot = True,
                                    save = False)

    # Predicting
    original, reconstructed = predict(model, data_prep[1], name, recurrent = False)

    # Save plots
    print(name.replace('df_', '') + ':')
    num_features = min(data_prep[1].shape[1], len(df.columns))
    mae, rmse, mape = save_predict_plots(original, reconstructed, df.columns, num_features, name, model_type = 'dense', plot = True, compute_df = False)
    mae_list.append(mae)
    rmse_list.append(rmse)
    mape_list.append(mape)
    name_list.append(name.replace('df_', ''))

    # Write metrics to csv
    metrics_data = {
        'Name': [name.replace('df_', '')],
        'MAE': [mae],
        'RMSE': [rmse],
        'MAPE': [mape]
    }
    metrics_df = pd.DataFrame(metrics_data)
    if os.path.exists(csv_file_path):
      metrics_df.to_csv(csv_file_path, mode = 'a', index = False, header = False)
    else:
      metrics_df.to_csv(csv_file_path, mode = 'w', index = False, header = True)

Epoch 1/500
1944/1944 [==============================] - 6s 3ms/step - loss: 0.0110 - val_loss: 0.0100
Epoch 2/500
1944/1944 [==============================] - 6s 3ms/step - loss: 0.0106 - val_loss: 0.0094
Epoch 3/500
1944/1944 [==============================] - 6s 3ms/step - loss: 0.0106 - val_loss: 0.0095
Epoch 4/500
1944/1944 [==============================] - 6s 3ms/step - loss: 0.0118 - val_loss: 0.0096
Epoch 5/500
1944/1944 [==============================] - 6s 3ms/step - loss: 0.0111 - val_loss: 0.0092
Epoch 6/500
1944/1944 [==============================] - 6s 3ms/step - loss: 0.0120 - val_loss: 0.0092
Epoch 7/500
1944/1944 [==============================] - 9s 5ms/step - loss: 0.0146 - val_loss: 0.0103
Epoch 8/500
1944/1944 [==============================] - 10s 5ms/step - loss: 0.0186 - val_loss: 0.0098
Epoch 9/500
1944/1944 [==============================] - 10s 5ms/step - loss: 0.0250 - val_loss: 0.0095
Epoch 10/500
1944/1944 [==============================] - 10s 5ms/step 

## Metrics

In [20]:
metrics = pd.read_csv('processing_metrics_dense.csv')
top_res = min(10, len(metrics))
print('Top ' + str(top_res) + ' MAE:')
mae_arr = metrics['MAE'].nsmallest(top_res)
for i in range(0, top_res):
    print(name_list[mae_arr.index[i]] + ': ' + str(mae_arr[mae_arr.index[i]]))
print('Top ' + str(top_res) + ' RMSE:')
rmse_arr = metrics['RMSE'].nsmallest(top_res)
for i in range(0, top_res):
    print(name_list[rmse_arr.index[i]] + ': ' + str(rmse_arr[rmse_arr.index[i]]))
print('Top ' + str(top_res) + ' MAPE:')
mape_arr = metrics['MAPE'].nsmallest(top_res)
for i in range(0, top_res):
    print(name_list[mape_arr.index[i]] + ': ' + str(mape_arr[mape_arr.index[i]]))

Top 10 MAE:
False_False_False_300_False_0_False_True: 1.467
True_False_False_300_False_0_False_True: 1.477
False_False_False_200_False_0_False_True: 1.702
True_False_False_200_False_0_False_True: 1.82
False_False_False_100_False_0_True_True: 1.829
False_False_False_300_False_0_True_True: 1.843
True_False_False_100_False_0_True_True: 2.075
True_False_False_200_False_0_True_True: 2.107
False_False_False_200_False_0_True_True: 2.111
True_False_False_100_False_0_False_True: 2.214
Top 10 RMSE:
False_False_False_300_False_0_False_True: 4.245
True_False_False_300_False_0_False_True: 4.54
False_False_False_100_False_0_True_True: 4.833
True_False_False_100_False_0_True_True: 5.409
True_False_False_100_False_0_False_True: 5.661
False_False_False_300_False_0_True_True: 5.773
False_False_False_200_False_0_False_True: 5.868
False_False_False_100_False_0_False_True: 6.17
True_False_False_200_False_0_True_True: 6.353
True_False_False_200_False_0_False_True: 6.446
Top 10 MAPE:
False_False_False_300_Fa

ratio_values = [True, False]
smoothing_values = [True, False]
collinearity_values = [True, False]
log_values = [True, False]
aggregation_values = [100, 200, 300]  
statistics_values = [True, False]
differencing_values = [0]
seasonality_values = [True, False]
normalize_values = [True]

Best combination(s): False/True_False_False_200/300_False_0_False_True

# Choosing preprocessing strategy - LSTM

## Preprocessing

In [26]:
ratio_values = [True, False]
smoothing_values = [True, False]
collinearity_values = [False]
log_values = [False]
aggregation_values = [100, 200, 300]  
statistics_values = [True, False]
differencing_values = [1]
seasonality_values = [True, False]
normalize_values = [True]

# Generate all combinations of parameter values
experiments = itertools.product(
    ratio_values, smoothing_values, collinearity_values, 
    log_values, aggregation_values, statistics_values, 
    differencing_values, seasonality_values, normalize_values
)

count = 1
tot_experiments = len(ratio_values) * len(collinearity_values) * len(log_values) * len(aggregation_values) * len(statistics_values) * len(differencing_values) * len(seasonality_values) * len(normalize_values)
warnings.filterwarnings("ignore", category = RuntimeWarning)
for combination in experiments:
    print(f"Experiment {count}/{tot_experiments}")
    print(f"ratio: {combination[0]}, smoothing: {combination[1]}, collinearity: {combination[2]}, log: {combination[3]}, aggregation: {combination[4]}, statistics: {combination[5]}, differencing: {combination[6]}, seasonality: {combination[7]}, normalize: {combination[8]}")
    df_path = ('df_' + str(combination[0]) + '_' + str(combination[1]) + '_' + str(combination[2]) + '_' + str(combination[3]) + '_' + str(combination[4]) + '_' + str(combination[5]) + '_' + str(combination[6]) + '_' + str(combination[7]) + '_' + str(combination[8]))
    if df_path not in df_path_lstm:
        df_path_lstm.append('datasets/proc/' + df_path + '.csv')
    if df_path not in df_path_dense and not os.path.exists('datasets/proc/' + df_path + '.csv'):
        df_ = copy.deepcopy(df)
        df_proc = preprocessing(df_,
                                ratio = combination[0],
                                smoothing = combination[1],
                                collinearity = combination[2],
                                log = combination[3],
                                aggregation = combination[4],
                                statistics = combination[5],
                                differencing = combination[6],
                                seasonality = combination[7],
                                normalize = combination[8],
                                load_scaler = False,
                                save = True)
    count += 1

Experiment 1/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: True, differencing: 1, seasonality: True, normalize: True
(67600, 248)
Experiment 2/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: True, differencing: 1, seasonality: False, normalize: True
(67600, 248)
Experiment 3/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: False, differencing: 1, seasonality: True, normalize: True
(67600, 31)
Experiment 4/24
ratio: True, collinearity: False, log: False, aggregation: 100, statistics: False, differencing: 1, seasonality: False, normalize: True
(67600, 31)
Experiment 5/24
ratio: True, collinearity: False, log: False, aggregation: 200, statistics: True, differencing: 1, seasonality: True, normalize: True
(67600, 248)
Experiment 6/24
ratio: True, collinearity: False, log: False, aggregation: 200, statistics: True, differencing: 1, seasonality: False, normalize: True
(67600, 248)
Experiment 7/24
rat

## Preparation

In [27]:
pattern = os.path.join('datasets/proc', 'df*.csv')
csv_files = glob.glob(pattern)
data_prep_dict = {}

count = 1
for file_path in csv_files:
    if file_path in df_path_lstm:
        print(str(count) + ': ' + file_path)
        filename = os.path.basename(file_path).replace('.csv', '')
        data_prep =  pd.read_csv(file_path)
        data_prep_dict[filename] = preparation(data_prep,
                                               window_size = 20,
                                               overlap = 0,
                                               val_hours = 1.5)
        count+=1

1: datasets/proc/df_True_False_False_200_True_1_False_True.csv
(3110, 20, 248)
(270, 20, 248)
2: datasets/proc/df_True_False_False_200_True_1_True_True.csv
(3110, 20, 248)
(270, 20, 248)
3: datasets/proc/df_False_False_False_300_True_1_True_True.csv
(3110, 20, 248)
(270, 20, 248)
4: datasets/proc/df_True_False_False_200_False_1_False_True.csv
(3110, 20, 31)
(270, 20, 31)
5: datasets/proc/df_False_False_False_300_False_1_True_True.csv
(3110, 20, 31)
(270, 20, 31)
6: datasets/proc/df_True_False_False_100_True_1_True_True.csv
(3110, 20, 248)
(270, 20, 248)
7: datasets/proc/df_True_False_False_100_False_1_True_True.csv
(3110, 20, 31)
(270, 20, 31)
8: datasets/proc/df_True_False_False_300_False_1_False_True.csv
(3110, 20, 31)
(270, 20, 31)
9: datasets/proc/df_True_False_False_200_False_1_True_True.csv
(900, 20, 31)
(270, 20, 31)
10: datasets/proc/df_False_False_False_100_False_1_False_True.csv
(3110, 20, 31)
(270, 20, 31)
11: datasets/proc/df_False_False_False_200_True_1_True_True.csv
(3110

## Training

In [28]:
mae_list = []
rmse_list = []
mape_list = []
name_list = []

csv_file_path = 'processing_metrics_lstm.csv'

for name, data_prep in data_prep_dict.items():
    # Training
    history, model = training_lstm(data_prep[0],
                                   data_prep[1],
                                   n_layers_encoder = 1,
                                   n_layers_decoder = 1,
                                   hidden_units = [128, 64, 64, 128],
                                   activation_hidden = 'relu',
                                   activation_dense = 'sigmoid',
                                   dropout_rate = 0.2,
                                   learning_rate = 0.001,
                                   n_epochs = 500,
                                   metric = 'mse',
                                   batch_size = 32,
                                   window_size = 20,
                                   plot = True,
                                   save = False)

    # Predicting
    original, reconstructed = predict(model, data_prep[1], name, recurrent = True)

    # Save plots
    print(name.replace('df_', '') + ':')
    num_features = min(data_prep[1].shape[2], len(df.columns))
    mae, rmse, mape = save_predict_plots(original, reconstructed, df.columns, num_features, name, model_type = 'lstm', plot = True, compute_df = False)
    mae_list.append(mae)
    rmse_list.append(rmse)
    mape_list.append(mape)
    name_list.append(name.replace('df_', ''))

    # Write metrics to csv
    metrics_data = {
        'Name': [name.replace('df_', '')],
        'MAE': [mae],
        'RMSE': [rmse],
        'MAPE': [mape]
    }
    metrics_df = pd.DataFrame(metrics_data)
    if os.path.exists(csv_file_path):
      metrics_df.to_csv(csv_file_path, mode = 'a', index = False, header = False)
    else:
      metrics_df.to_csv(csv_file_path, mode = 'w', index = False, header = True)

Epoch 1/500
98/98 [==============================] - 7s 54ms/step - loss: 0.0088 - val_loss: 0.0043
Epoch 2/500
98/98 [==============================] - 6s 62ms/step - loss: 0.0043 - val_loss: 0.0037
Epoch 3/500
98/98 [==============================] - 6s 61ms/step - loss: 0.0036 - val_loss: 0.0035
Epoch 4/500
98/98 [==============================] - 6s 62ms/step - loss: 0.0032 - val_loss: 0.0032
Epoch 5/500
98/98 [==============================] - 6s 63ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 6/500
98/98 [==============================] - 6s 61ms/step - loss: 0.0028 - val_loss: 0.0030
Epoch 7/500
98/98 [==============================] - 6s 61ms/step - loss: 0.0027 - val_loss: 0.0030
Epoch 8/500
98/98 [==============================] - 6s 58ms/step - loss: 0.0026 - val_loss: 0.0029
Epoch 9/500
98/98 [==============================] - 6s 61ms/step - loss: 0.0025 - val_loss: 0.0029
Epoch 10/500
98/98 [==============================] - 6s 58ms/step - loss: 0.0025 - val_loss: 0.0029

## Metrics

In [30]:
metrics = pd.read_csv('processing_metrics_lstm.csv')
top_res = min(10, len(metrics))
print('Top ' + str(top_res) + ' MAE:')
mae_arr = metrics['MAE'].nsmallest(top_res)
for i in range(0, top_res):
    print(name_list[mae_arr.index[i]] + ': ' + str(mae_arr[mae_arr.index[i]]))
print('Top ' + str(top_res) + ' RMSE:')
rmse_arr = metrics['RMSE'].nsmallest(top_res)
for i in range(0, top_res):
    print(name_list[rmse_arr.index[i]] + ': ' + str(rmse_arr[rmse_arr.index[i]]))
print('Top ' + str(top_res) + ' MAPE:')
mape_arr = metrics['MAPE'].nsmallest(top_res)
for i in range(0, top_res):
    print(name_list[mape_arr.index[i]] + ': ' + str(mape_arr[mape_arr.index[i]]))

Top 10 MAE:
True_False_False_300_False_1_False_True: 0.003
False_False_False_300_False_1_False_True: 0.003
True_False_False_200_False_1_False_True: 0.005
False_False_False_200_False_1_False_True: 0.005
False_False_False_100_False_1_False_True: 0.008
True_False_False_100_False_1_False_True: 0.01
False_False_False_100_False_1_True_True: 0.02
False_False_False_300_False_1_True_True: 0.032
True_False_False_200_False_1_True_True: 0.042
True_False_False_300_False_1_True_True: 0.046
Top 10 RMSE:
True_False_False_300_False_1_False_True: 0.008
False_False_False_300_False_1_False_True: 0.009
True_False_False_200_False_1_False_True: 0.012
False_False_False_200_False_1_False_True: 0.013
False_False_False_100_False_1_False_True: 0.021
True_False_False_100_False_1_False_True: 0.023
False_False_False_100_False_1_True_True: 0.078
False_False_False_300_False_1_True_True: 0.16
True_False_False_200_False_1_True_True: 0.223
True_False_False_100_False_1_True_True: 0.227
Top 10 MAPE:
True_False_False_300_Fa

ratio_values = [True, False]
smoothing_values = [True, False]
collinearity_values = [True, False]
log_values = [False]
aggregation_values = [100, 200, 300]  
statistics_values = [True, False]
differencing_values = [1, 100]
seasonality_values = [True, False]
normalize_values = [True]

Best combination(s): False/True_False_False_200/300_False_1_False_True